In [ ]:
from __future__ import print_function, division, absolute_import
import os,sys
import numpy as np
from scipy.linalg import inv, block_diag
import time
from joblib import Parallel, delayed
import gudhi
sys.path.insert(0, "C:\\Users\\matt\\Desktop\\orbithunter\\")
from orbithunter import *
from mpl_toolkits.mplot3d.axes3d import *
import matplotlib.pyplot as plt
import itertools
from gudhi.hera import wasserstein_distance, bottleneck_distance
import glob
import time
import h5py
import gudhi
import os
import orbithunter.persistent_homology as ohgudhi
import gudhi
from orbithunter.shadowing import pivot_iterator, scanning_dimensions
from orbithunter.persistent_homology import orbit_persistence_array, persistence_array_converter, slice_persistence_array
import orbithunter as oh
score_type='persistence'
scoring_function = ohgudhi.gudhi_distance_from_persistence
persistence_function = ohgudhi.orbit_persistence

# Identical diagrams between group orbit members.

In [ ]:
# persistences = []
# sr = read_h5('../../data/ShiftReflectionOrbitKS_L34p465_T30p451.h5')
# group_orbit = [sr, sr.reflection(), sr.cell_shift(axis=1), sr.cell_shift().reflection()]
# for orbit in group_orbit:
#     orbit.plot(fundamental_domain=False)
# #     pcc = orbit_periodic_cubical_complex(orbit)
#     ps = orbit_persistence(orbit, min_persistence=0.5)
#     persistences.append([p[-1] for p in ps])
#     gudhi_plot(ps, legend=True)
    
# for g in group_orbit:
#     print(gudhi_distance(group_orbit[0], g, metric='wasserstein'))

# orbit_list = []
# for orbit_h5 in glob.glob('../../data/local/continuation/merge_tile_coarse/*.h5'):
#     orbit_list.append(read_h5(os.path.basename(orbit_h5), directory='../data/local/continuation/merge_tile_coarse/'))

# orbit1 = read_h5('RelativeOrbitKS_L13p093_T9p100.h5', directory='../data/local/continuation/merge_tile/')
# orbit2 =  read_h5('RelativeOrbitKS_L12p997_T22p702.h5', directory='../data/local/continuation/merge_tile/')

# from orbithunter.gluing import tile_dictionary_ks
# td = tile_dictionary_ks(padded=False, comoving=False)

# td[0].plot()

# td[1].plot()

# td[2].plot()

# tile_dictionary = rediscretize_tiling_dictionary(td, new_shape=(16,16))

# orbit1.plot()

# orbit2.plot()

# pcc1 = orbit_persistence(orbit1, min_persistence=0.5)
# pcc2 = orbit_persistence(orbit2, min_persistence=0.5)

# gudhi_plot(pcc1)

# gudhi_plot(pcc2)

# wd_list_bd = []
# for orbit1, orbit2 in np.array(list(itertools.product(orbit_list, orbit_list))):
#     wd_list_bd.append(gudhi_distance(orbit1, orbit2, metric='bottleneck'))

# plt.matshow(np.reshape(wd_list_bd, (int(np.sqrt(len(wd_list_bd))), -1)), cmap='jet')
# plt.colorbar()
# plt.show()

# The block diagonal structure is an indication that orbits indexed from 0 ~ 35 are 'closer' to each other than those indexed
# from 35-...

# Possible applications: bifurcation detection, identifying orbits, symbolic dynamics detection.

# for o in orbit_list[35:]:
#     o.plot()
# # orbit_list[35].plot(), orbit_list[-1].plot()

# orbit_list[35].plot(), orbit_list[40].plot()

# Identification and classification of continuous families.

In [1]:
# window = window_orbit
# strides = (1, 1)
# scanning_shapes = scanning_dimensions(base.shape, window.shape, strides, (False, True))

# # base_orbit_persistence = orbit_persistence_array(base, window, strides, scanning_shapes, persistence_function, verbose=True)

# window_persistence = persistence_function(window_orbit, **gudhikwargs)
# window_persistence_abs = persistence_function(window_orbit.abs(), **gudhikwargs)

# window_persistence = persistence_function(window_orbit, **gudhikwargs)
# plt.figure()
# for metric in ['bottleneck', 'hera_bottleneck', 'wasserstein']:
#     gudhikwargs['gudhi_metric'] = metric
#     tp0= time.time_ns()/10**9
#     for i, bp in enumerate(base_persistences):
#         score_array.ravel()[i] = scoring_function(bp, window_persistence, **gudhikwargs)
#     tp1= time.time_ns()/10**9
#     print(tp1-tp0)
#     plt.hist(score_array.ravel(), bins=25, alpha=0.5)
# plt.show()

# # list_of_persistence_arrays = [np.concatenate([np.concatenate(([y[0]], y[-1])) 
# #                                               for y in base_cache[pivot_label]]).reshape(-1,3)
# #                                  for pivot_label in base_cache]


# # with h5py.File('large_patch_persistence.h5', 'w') as file:
# #     new_persistence_array = []
# #     for pivot_tuple, persistence_arr in zip(base_cache, list_of_persistence_arrays):
# #         left_pivot = np.array([pivot_tuple[0]]*persistence_arr.shape[0])
# #         right_pivot = np.array([pivot_tuple[1]]*persistence_arr.shape[0])
# #         new_persistence_array.append(np.concatenate((left_pivot.reshape(-1,1),
# #                                                      right_pivot.reshape(-1,1),
# #                                                      persistence_arr),axis=1))
# #     new_persistence_array=np.concatenate(new_persistence_array)
# #     file.create_dataset(lab, data=new_persistence_array)
# #     print('.', end='')
    